In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
import os
import joblib

### Initial Data Inspection

In [2]:
df=pd.read_csv('Data_Sets\SuperMarket Analysis.csv')

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20428\790930629.py:1: SyntaxWarning: invalid escape sequence '\S'
  df=pd.read_csv('Data_Sets\SuperMarket Analysis.csv')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20428\790930629.py:1: SyntaxWarning: invalid escape sequence '\S'
  df=pd.read_csv('Data_Sets\SuperMarket Analysis.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'Data_Sets\\SuperMarket Analysis.csv'

In [ ]:
df.head()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.duplicated().sum()

In [ ]:
df.info()

### Feature Engineering on Date/Time Columns

In [ ]:
df['Date']=pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], format='%I:%M:%S %p').dt.time


In [ ]:
df['Date'].head()

In [ ]:
df['Time'].head()

In [ ]:
df.describe()

In [ ]:
# ML Models cannot use raw date and time data directly do we convert them into numerical columns

In [ ]:
df.head()

In [ ]:
# We can also map the values if necassary

In [ ]:
df['DayOfWeek'] = df['Date'].dt.dayofweek


In [ ]:
df['Day'] = df['Date'].dt.day


In [ ]:
df['Month'] = df['Date'].dt.month


In [ ]:
#  Fot Time

In [ ]:
# df['Hour'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour
df['Hour'] = df['Time'].apply(lambda x: x.hour)


In [ ]:
df['TimeOfDay'] = df['Hour'].apply(
    lambda h: 'Morning' if 5 <= h < 12 else
              'Afternoon' if 12 <= h < 17 else
              'Evening' if 17 <= h < 21 else
              'Night'
)


In [ ]:
df.head()

In [ ]:
df.drop(columns=['Date','Time'],inplace=True)

In [ ]:
df.drop(columns=['Invoice ID'])

## EDA

##### Univariate Analysis

In [ ]:
num_cols=df.select_dtypes(include=['number','int','float'])
cat_cols=df.select_dtypes(exclude=['number','int','float'])

In [ ]:
num_cols.columns

In [ ]:
for col in num_cols:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.show()

In [ ]:
from scipy.stats import skew
print("Skewness:", skew(num_cols['Sales']))

In [ ]:
# Skewness 0.8912303920037631 means that the target column is right skewed, hence we use log transformation to make it normal

In [ ]:
# Will do this after initail model trianing IF NECESSARY
# df['Sales_transformed'] = np.log1p(num_cols['Sales'])

In [ ]:
# from scipy.stats import skew
# print(skew(train_cleaned['Sales_transformed']))

In [ ]:
# using powertransformer to handle skeweness

In [ ]:
#Skip for now
# from sklearn.preprocessing import PowerTransformer

# pt = PowerTransformer(method='yeo-johnson')
# train_cleaned['Sales_transformed'] = pt.fit_transform(num_cols[['Sales']])


In [ ]:
# plt.figure(figsize=(5,6))
# sns.histplot(train_cleaned['Sales_transformed'], kde=True)
# plt.title(f'Distribution of {col}')
# plt.show()

In [ ]:
num_cols=df.select_dtypes(include=['int','float','number'])

In [ ]:
for col in num_cols:
    plt.figure(figsize=(5,6))
    sns.boxplot(num_cols[col])
    plt.title(col)
    plt.show()

In [ ]:
# We can see that there are outliers present in the target column

In [ ]:
# import numpy as np

# Q1 = num_cols['Sales_transformed'].quantile(0.25)
# Q3 = num_cols['Sales_transformed'].quantile(0.75)
# IQR = Q3 - Q1

# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# outliers = num_cols[(num_cols['Sales_transformed'] < lower_bound) | (num_cols['Sales_transformed'] > upper_bound)]
# print(f"Outlier count: {len(outliers)} ({len(outliers)/len(num_cols)*100:.2f}% of data)")


In [ ]:
# To deal with outlier we use Winsorization

# Winsorization is a statistical technique used to limit the influence of extreme values (outliers)
#  by capping them at specific percentile thresholds — rather than removing them.
# It brings extreme values closer to the bulk of the data, instead of deleting them.


In [ ]:
#Skip for now
# from scipy.stats.mstats import winsorize
# train_cleaned['Sales_transformed_capped']=winsorize(train_cleaned['Sales_transformed'],limits=[0.01,0.01])


In [ ]:
cat_cols.head()

In [ ]:
for col in cat_cols:
    vc = df[col].value_counts() # counts how many times each unique category appears in that column.
    pct = df[col].value_counts(normalize=True).mul(100)#finds what percentage of total data each represents.
    summary = pd.concat([vc, pct], axis=1)# axis=1 means to concatenate horizontally
    summary.columns = ['count','percent']
    print(f"\n=== {col} ===")
    print(summary)


In [ ]:
cat_cols.head()

In [ ]:

for col in cat_cols:
    sns.countplot(data=df,x=col)
    plt.title(f'Distribution of {col}')
    plt.xticks(rotation=45)
    plt.show()

-Bivariate Data

In [ ]:
# -----------------------------
# 1. Numerical vs Numerical
# -----------------------------
# Scatterplots for all numeric column pairs
num_columns = num_cols.columns

for i in range(len(num_columns)):
    for j in range(i+1, len(num_columns)):
        plt.figure(figsize=(6,4))
        sns.scatterplot(x=df[num_columns[i]], y=df[num_columns[j]])
        plt.title(f'{num_columns[i]} vs {num_columns[j]}')
        plt.xlabel(num_columns[i])
        plt.ylabel(num_columns[j])
        plt.show()

# Correlation Heatmap
plt.figure(figsize=(8,6))
sns.heatmap(df[num_columns].corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap for Numerical Variables")
plt.show()


# -----------------------------
# 2. Numerical vs Categorical
# -----------------------------
cat_columns = cat_cols.columns

for cat in cat_columns:
    for num in num_columns:
        plt.figure(figsize=(6,4))
        sns.boxplot(x=df[cat], y=df[num])
        plt.title(f'{num} by {cat}')
        plt.xlabel(cat)
        plt.ylabel(num)
        plt.xticks(rotation=45)
        plt.show()


# -----------------------------
# 3. Categorical vs Categorical
# -----------------------------
for i in range(len(cat_columns)):
    for j in range(i+1, len(cat_columns)):
        ctab = pd.crosstab(df[cat_columns[i]], df[cat_columns[j]])

        plt.figure(figsize=(6,4))
        sns.heatmap(ctab, annot=True, fmt='d', cmap='Blues')
        plt.title(f'{cat_columns[i]} vs {cat_columns[j]}')
        plt.xlabel(cat_columns[j])
        plt.ylabel(cat_columns[i])
        plt.xticks(rotation=45)
        plt.yticks(rotation=45)
        plt.show()

Regression Line


In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Assume train_processed has been preprocessed
# numeric_cols = df.select_dtypes(include=['float64','int64']).columns

# # Compute correlation matrix
# corr_matrix = df[numeric_cols].corr()

# #Regression line plot
# sns.lmplot(x='Item_MRP', y='Sales_transformed_capped', data=df, height=6)
# plt.title("Linear Regression Line: MRP vs Sales")
# plt.show()



Scatter Plot


In [ ]:
# plt.figure(figsize=(8,6))
# sns.scatterplot(x='Item_MRP', y='Sales_transformed_capped', data=train_cleaned)
# plt.title("Item MRP vs Sales")
# plt.show()


Box plot


To see the sales in outlet location types


In [ ]:
# categorical_cols = train_cleaned.select_dtypes(include=['object']).columns

# # Example: How Outlet_Location_Type affects Sales
# plt.figure(figsize=(8,6))
# sns.boxplot(x='Outlet_Location_Type', y='Sales_transformed_capped', data=train_cleaned)
# plt.title("Outlet Type vs Sales")
# plt.show()

In [ ]:
# plt.figure(figsize=(5,6))
# sns.barplot(x='Item_Type', y='Sales_transformed_capped', data=train_cleaned)
# plt.show()

In [ ]:
num_cols.columns

In [ ]:

plt.figure(figsize=(10,6))
sns.heatmap(num_cols.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap of Numerical Features")
plt.show()

### Feature Engineering

In [ ]:
# # Item Category
# # Extract first two letters from Item_ID to classify into Food/Drinks/Non-Consumable

# combined['Item_Category'] = combined['Item_ID'].str[:2].map({
#     'FD': 'Food',
#     'DR': 'Drinks',
#     'NC': 'Non-Consumable'
# })


# # Interaction Feature
# # IT refers to new features created by combining two or more existing features in a way that captures their relationship or interaction
# # Numerical Columns
# # 1. Relationship between item price and weight
# combined['MRP_Weight'] = combined['Item_MRP'] * combined['Item_W']

# #2.Item MRP × Sales:
# combined['MRP_Sales_interaction'] = combined['Item_MRP'] * combined['Sales']

# # 3. Item MRP × Outlet Year
# combined['MRP_OutletYear_interaction'] = combined['Item_MRP'] * combined['Outlet_Year']

# #4. Sales × MRP Weight

# combined['Sales_MRPWeight_interaction']= combined['Sales'] * combined['MRP_Weight']

# # Categorical Columns
# # -------------------------------------------------------------------
# #Research Polynomial Features

In [ ]:
# # fitting on train and transforming test separately may be okay for label encoding,
# # but doing all encodings on `combined` ensures absolute consistency for all categorical transforms.
# # Label-encode 'Outlet_ID' on combined so encoding is consistent across train/test
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# combined['Outlet_ID'] = le.fit_transform(combined['Outlet_ID'].astype(str))

# # Now that we've applied all encodings on `combined`, re-split into train_cleaned and test_cleaned
# # This ensures the encoded columns appear exactly the same in both sets.
# train_cleaned = combined[combined['source'] == 'train'].copy()
# test_cleaned = combined[combined['source'] == 'test'].copy()

# # If 'Sales_transformed_capped' was previously created for modeling, ensure it exists for train
# # If not present yet, create it by copying 'Sales' (this was your previous simple approach)
# if 'Sales_transformed_capped' not in train_cleaned.columns:
#     train_cleaned['Sales_transformed_capped'] = train_cleaned['Sales']

# # Drop the helper 'source' column now that we've re-split (keeps dataframes clean)
# train_cleaned.drop(columns=['source'], inplace=True)
# test_cleaned.drop(columns=['source'], inplace=True)


In [ ]:
# # Create output folder, fit the chosen model on the full training data, and save model + preprocessor
# import os
# from sklearn.ensemble import GradientBoostingRegressor

# os.makedirs('ML_Model', exist_ok=True)

# # Instantiate the model
# best_model = GradientBoostingRegressor(random_state=42)

# # Fit on the full preprocessed training data
# # X_train_processed and y_train are expected to be in the notebook kernel
# best_model.fit(X_train_processed, y_train)

# # Save the fitted model
# import joblib
# joblib.dump(best_model, os.path.join('ML_Model', 'GradientBoostingRegressor.pkl'))
# print("Saved fitted GradientBoostingRegressor to ML_Model/GradientBoostingRegressor.pkl")

# # Also save the preprocessing pipeline so you can transform new data the same way
# try:
#     joblib.dump(preprocessor, os.path.join('ML_Model', 'preprocessor.pkl'))
#     print("Saved preprocessor to ML_Model/preprocessor.pkl")
# except Exception as e:
#     print("Warning: could not save preprocessor:", e)


In [ ]:
# # TARGET & DATA DIAGNOSTICS
# print("=== Target (Sales) Diagnostics ===")
# print("Target statistics:")
# print(y_train.describe())
# print("\nTarget distribution skewness:", y_train.skew())
# print("\nFeature count:", X_train_processed.shape[1])
# print("Train samples:", X_train_processed.shape[0])
# print("Target variance:", y_train.var())
# print("Target std dev:", y_train.std())



In [ ]:
# # Check if target has unusual properties
# print("\nSales value counts (bottom 10):")
# print(y_train.value_counts().tail(10))
# print("\nUnique sales values:", y_train.nunique())

In [ ]:
# # Feature importance via correlation matrix
# print("\n=== Feature Summary ===")
# print("Features that will be used:")
# for i, feat in enumerate(numeric_features):
#     print(f"  {i+1}. {feat}")

### Encoding


In [ ]:
num_cols.head()

In [ ]:
# Encoding numerical columns with standard scalar
num_cols = num_cols.columns.tolist()
scaler = StandardScaler()
df_scaled = df.copy()  # keep original dataset
df_scaled[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
# cat_cols.drop(columns=["Invoice ID"],inplace=True)

In [ ]:
cat_cols.head()

In [ ]:
# Encoding categorical columns with One Hpt Encoding
#Note Customer Type might need to be encoded in another way
cat_cols = cat_cols.columns.tolist()
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=False)


In [ ]:
# df_encoded.drop(columns=["Invoice ID"],inplace=True)

In [ ]:
df_encoded.head()

### Initial Model Training

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

X = df_encoded.drop(columns=["Sales"])  # features
y = df_encoded["Sales"]                 # target


X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,     # 20% test, 80% train
    random_state=42,   # reproducible
)


models =  {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "SVR": SVR(kernel="rbf")
}


for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"=== {name} ===")
    print("R² Score:", r2_score(y_test, y_pred))
    print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
    print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
    print("\n")

